# 基于图卷积神经网络t-SNE共识表示学习的非完整多视角特征补全的


## 网络结构

![image.png](image-small.png)

网络结构图

### 网络输入 Network Inputs

- $ \tilde{X}^{(v)} \in R^{ n^{(v)} \times d^{(v)} } $ 为每个视角的存在样本；
- $ {X}^{(v)} \in R^{ n \times d^{(v)} } $ 为每个视角的所有样本（不存在的样本标记为NaN）；
- $M^{(v)} \in \{0,1\}^{n^{(v)} \times n} $ 为缺失指示矩阵；
- $  n^{(v)}, d^{(v)} $ 分别为每个视角的存在样本数量和每个视角的原始特征维度；
- $ n, V $ 分别为样本数量（总数量，包括缺失和非缺失的）和视角数量；

$$
 M^{(v)}_{i,j} = \begin{cases}
    1,\text{ if sample $j$ is the $i$-th sample that exists,} \\
    0,\text{ otherwise}
 \end{cases}
$$

$$
\tilde{X}^{(v)} = M^{(v)} {X}^{(v)}
$$

### 网络输出 Network Outputs

- $ \hat{X}^{(v)} \in R^{ n^{(v)} \times d^{(v)} } $ 是网络为每个视角重建的原始特征，其中包含对存在样本的重建和对不存在样本（缺失样本）的预测（补全）；
- $H \in R^{n \times d} $ 是网络学习到的共识表示，可用于聚类等其他下游任务，$d$ 是共识表示的维度；

## 共识表示学习模块 Consensus Representation Learning Module

### 构图 Graph Construction

- 从 $\tilde{X}^{(v)}$ 中构建 $\tilde{P}^{(v)}, \tilde{A}^{(v)} \in R^{ n^{(v)} \times n^{(v)} } $；
- $\tilde{P}^{(v)}$ 是视角v的存在样本的原始数据分布，是高斯分布，用作t-SNE损失函数的监督信号；
- $\tilde{A}^{(v)}$ 是视角v的存在样本的归一化的相似度图，用作GCN的图输入；
- $\tilde{S}^{(v)}$ 是视角v的存在样本的未归一化相似度图（基于高斯核，也可以基于其他核函数）；
- $\tilde{D}^{(v)} \in R^{ n^{(v)} \times n^{(v)} } $ 是对角矩阵，用于对 $\tilde{A}^{(v)}$ 进行归一化；
- $I_{n^{(v)}} \in R^{ n^{(v)} \times n^{(v)} }$ 是单位矩阵，用于使得图包含自环；
- $\sigma_i$ 是根据困惑度PPL（模型的超参数）利用二分查找求出；


$$
\tilde{S}_{i,j} = \frac{ \exp\left(-||\tilde{x}_i - \tilde{x}_j||^2 / 2 \sigma_i^2
\right)}{\sum_{k\neq i}\exp\left(-||\tilde{x}_i - \tilde{x}_k||^2 / 2 \sigma_i^2\right)}
$$

$$
\tilde{P}^{(v)} = \frac{1}{2} \left(\tilde{S}^{(v)} + (\tilde{S}^{(v)})^T\right)
$$

$$
\tilde{A}^{(v)} = (\tilde{D}^{(v)})^{-1/2} \left( \tilde{S}^{(v)} + I_{n^{(v)}} \right) (\tilde{D}^{(v)})^{-1/2}
$$

$$
\tilde{D}_{i,i}^{(v)} = \sum_{j=1}^{n^{(v)}} \tilde{S}_{i,j}^{(v)}
$$

### 困惑度超参数

- 困惑度Perplexity（PPL），决定了原始特征的每个样本周围有多少邻居；
- $H(P)$ 表示分布$P$的熵；
- $P_i$ 表示$i$出现的概率；
- 根据设定的PPL，用二分搜索得到对应的$\sigma$；
  - 每给定一个$\sigma$，就得到一个分布$P$；
  - 由此计算到$H(P)$和PPL；
  - 判断两个PPL是否相等，不等则继续搜索；

$$
PPL(P) = 2^{H(P)}
$$

$$
H(P) = -\sum_{i} P_{i} \log_2 P_{i}
$$

### 视角专属编码器 View-specific Encoder

- $ \tilde{H}^{(v)} \in R^{n^{(v)} \times {d^{(v)}}} $ 是编码器输出的特征
- $ f^{(v)} $ 是编码器，$ \Theta_E^{(v)} $ 是编码器的可学习参数；
- $ H \in R^{n \times d} $ 是共识表示，$d$ 是共识表示的维度；
- $ \bar{H}^{(v)} $ 是共识表示到各视角的投影；
- 编码器由图卷积层和批归一化层组成；

$$
\tilde{H}^{(v)} = f^{(v)}\left(\tilde{X}^{(v)}, \tilde{A}^{(v)}; \Theta_E^{(v)}\right)
$$

$$
H = \sum_{v=1}^V (M^{(v)})^T \tilde{H}^{(v)} \\
$$

$$
\bar{H}^{(v)} = M^{(v)} H
$$

### 图卷积层 Graph Convolutional Layer

- $ReLU(x) = \max(0, x)$ ；
- $H^{(l)}$ 是网络第$l$层的输入，$H^{(0)} = X$ 是网络的输入；
- $H^{(l+1)}$ 是网络第$l$层的输出，也是第$l+1$层的输入；
- $\tilde{A}$ 是归一化的相似度图，用作GCN的图输入；
- $W^{(l)}$ 是网络第$l$层的可学习参数；

$$
H^{(l+1)} = ReLU \left( \tilde{A} H^{(l)} W^{(l)} \right)
$$

### 损失函数 Loss Function
- $ \bar{Q}^{(v)} \in R^{ n^{(v)} \times n^{(v)} } $ 是从 $ \bar{H}^{(v)} $ 构造的分布，是自由度为1的t分布；
- $ KL(\cdot||\cdot) $ 是 Kullback-Leibler(KL)散度；


$$
\bar{Q}_{i,j}^{(v)} = \frac{
    \left(1 + || \bar{h}_i^{(v)} - \bar{h}_j^{(v)} ||^2\right)^{-1}
}{
    \sum_{k\neq l} \left(1 + || \bar{h}_k^{(v)} - \bar{h}_i^{(v)} ||^2\right)^{-1}
}
$$

$$
\mathcal{L}_{t-SNE} = \sum_{v=1}^V KL\left( \tilde{P}^{(v)} ||\ \bar{Q}^{(v)} \right)
$$

## 缺失视角特征补全模块 Feature Completion Module of Missing Views

### 视角专属解码器 View-specific Decoder

- $ \hat{X}^{(v)} \in R^{ n^{(v)} \times d^{(v)} } $ 是网络为每个视角重建的原始特征，其中包含对存在样本的重建和对不存在样本（缺失样本）的预测（补全）；
- $ g^{(v)} $ 是解码器，$ \Theta_D^{(v)} $ 是解码器的可学习参数；
- $ H \in R^{n \times d} $ 是共识表示，$d$ 是共识表示的维度；
- 解码器由全连接层和批归一化层组成；


$$
\hat{X}^{(v)} = g^{(v)}\left(H; \Theta_D^{(v)}\right) \\
$$

### 全连接层 Full-connected Layer

- $ReLU(x) = \max(0, x)$ ；
- $ H^{(l)} $ 是网络第$l$层的输入，$H^{(0)} = X$ 是网络的输入；
- $H^{(l+1)}$ 是网络第$l$层的输出，也是第$(l+1)$层的输入；
- $ W^{(l)}, b^{(l)} $ 是网络第$l$层的可学习参数；

$$
H^{(l+1)} = ReLU \left( H^{(l)} W^{(l)} + b^{(l)} \right)
$$

### 损失函数 Loss Function

- $ M^{(v)} \in \{0,1\}^{n^{(v)} \times n} $ 为缺失指示矩阵；
- $ \hat{X}^{(v)} \in R^{ n^{(v)} \times d^{(v)} } $ 是网络为每个视角重建的原始特征，其中包含对存在样本的重建和对不存在样本（缺失样本）的预测（补全）；
- $ {X}^{(v)} \in R^{ n \times d^{(v)} } $ 为每个视角的所有样本（不存在的样本标记为NaN）；


$$
\mathcal{L}_{MSE} = \sum_{v=1}^V \frac{1}{n^{(v)}} \left( M^{(v)}|| \hat{X}^{(v)} - {X}^{(v)} ||^2 \right)
$$

### 总体损失函数 Overall Loss Function

- $ \mathcal{L}_{t-SNE} , \mathcal{L}_{MSE} $ 分别是共识表示学习损失和特征补全损失；
- $\lambda > 0$ 是权衡上述两项损失函数的超参数；
- 模型参数为：$\Theta = \{\Theta^{(v)}_E,\Theta^{(v)}_D\}_{v=1}^{V} $；

$$
\mathcal{L} = \mathcal{L}_{t-SNE} + \lambda \cdot \mathcal{L}_{MSE}
$$

$$
\mathcal{L} = \sum_{v=1}^V KL\left( \tilde{P}^{(v)} ||\ \bar{Q}^{(v)} \right) + \lambda \cdot \frac{1}{n^{(v)}} \left( M^{(v)}|| \hat{X}^{(v)} - {X}^{(v)} ||^2 \right)
$$

## 实现细节 Implementation Details

- 使用 Adam 优化器，学习率设置为0.0001；
- 数据预处理：将每个特征归一化为0-1之间的实数；
- 编码器的结构如下：
  - 图卷积层-1
  - 批归一化层-1
  - 图卷积层-2
  - 批归一化层-2
- 解码器的结构如下：
  - 批归一化层-3
  - 全连接层-1
  - 批归一化层-4
  - 全连接层-2
  - Sigmoid层
- 超参数：
  - $\lambda$ 设置为0.1；
  - 困惑度PPL设置为50;